In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
import numpy as np
    
def token_lookup():
    token_dict = {".":"<PERIOD>", ",": "<COMMA>", ";":"<SEMICOLON>", ":":"<COLON>", "\"":"<QOUTATION>", "'":"QOUTATION", "!":"<EXCLAMATION>", "?":"<QUESTION>", 
                  "(": "<LEFTPARAN>", ")": "<RIGHTPARAN>", "{":"LEFTBRACE", "}":"<RIGHTBRACE>", "[":"<LEFTBRACKET>", "]":"<RIGHTBRACKET>", "--":"<DASH>", "-":"<HYPHEN>",
                  "\n":"<RETURN>"}
    return token_dict

token_dict = token_lookup()

english_text = open("./plot.tok.gt9.5000").read()#open("./plot_quote.10000").read()

for key, token in token_dict.items():
    english_text = english_text.replace(key, ' {} '.format(token))
english_text = english_text.lower()
english_text = english_text.split()
english_text = ' '.join(english_text)

english_text = english_text.split("<period>")
english_text = pd.DataFrame(english_text, columns=["sentences"])

#hindi_text = pd.read_csv("./hindi_train.csv").drop(columns=['experience']).iloc[:50]
mix_text = open("./codemix.txt").read()

for key, token in token_dict.items():
    mix_text = mix_text.replace(key, ' {} '.format(token))
mix_text = mix_text.lower()
mix_text = mix_text.split()
mix_text = ' '.join(mix_text)

mix_text = mix_text.split("<period>")
mix_text = pd.DataFrame(mix_text, columns=["sentences"])
mix_text.head

<bound method NDFrame.head of                                                sentences
0      the biggest drawback of इस योजना is that इस सब...
1                 <return> दिल्ली is an urban territory 
2       <return> there banks are in बैंक बहुतायत and ...
3       <return> यही बात can also be said about आधार ...
4       <return> but various complaints are आ रही हैं...
...                                                  ...
26394                         <return> give सबको खींचके 
26395   <return> क्यों krishna हमें क्यों slap देंगी ...
26396                   <return> लेकिन today फिर i lost 
26397                 <return> you मुझसे तकलीफ़ नहीं है 
26398                <return> you तकलीफ़ इस बात <return>

[26399 rows x 1 columns]>

In [4]:
en_mix = pd.concat([english_text, mix_text], axis=0)
en_mix = en_mix.sample(frac = 1)
en_mix.head()

,sentences
13973,<return> but who did force तुझे to be a statu...
381,<return> lauren hynde <leftparan> shannyn sos...
24101,<return> काले कीड़े की भाँति black insect मूल...
5555,<return> सर्वोत्तम समय for replanting is from...
18925,<return> some plants सेमी शेड की require होती...


In [5]:
def create_lookup_tables(text_df):
    vocab = set()
    for line in text_df['sentences']:
        vocab.update(set(line.split()))
    #vocab = set(text)
    vocab_to_int = {c: i for i, c in enumerate(vocab)}
    int_to_vocab = dict(enumerate(vocab))
    return (vocab_to_int, int_to_vocab)

vocab_to_int, int_to_vocab = create_lookup_tables(en_mix)

train_int_text = []

for line in en_mix['sentences']:
    train_int_text.append([vocab_to_int[word] for word in line.split()])

train_int_text = [item for sublist in train_int_text for item in sublist]
print(train_int_text[:1000])

[6504, 11206, 13823, 5912, 535, 742, 19279, 11062, 16472, 27180, 20683, 6504, 28452, 30118, 12425, 33929, 18548, 30118, 13823, 30352, 30134, 30396, 19279, 11145, 28284, 1731, 20283, 32195, 12425, 20283, 32264, 6504, 36651, 30545, 12676, 32133, 7540, 36591, 5938, 2185, 16472, 22239, 6504, 22824, 36273, 19144, 35439, 11391, 13640, 22835, 30118, 12315, 30118, 8470, 35591, 10700, 32432, 9820, 34077, 12253, 289, 203, 36074, 7187, 6504, 34715, 8819, 29675, 27494, 5938, 1731, 20283, 1095, 36927, 12425, 11493, 19279, 6022, 12425, 3183, 29416, 6504, 1274, 12253, 10688, 27937, 19144, 36590, 11072, 7187, 6504, 20283, 7075, 20283, 9090, 4650, 35591, 5361, 21060, 18573, 35583, 11072, 7187, 6504, 7481, 4493, 34415, 20283, 3271, 1731, 7234, 26567, 7848, 21222, 17173, 4202, 17192, 12425, 21222, 29816, 13198, 4493, 11062, 20644, 34296, 6482, 7187, 20114, 21172, 12425, 16749, 13179, 36565, 25521, 6504, 27432, 17644, 8493, 8487, 18459, 30661, 30203, 1293, 20283, 20811, 6690, 25119, 3234, 34995, 1731, 226

In [6]:
print(en_mix.shape, len(vocab_to_int), max(train_int_text))

(31787, 1) 37262 37261


In [7]:
#import helper

#data_dir = './brown.txt'

#helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

from distutils.version import LooseVersion
import warnings
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import tensorflow as tf2
#import helper
import numpy as np

#int_text, val_int_text, test_int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

def get_inputs():
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learn_rate = tf.placeholder(tf.float32, name='learn_rate')
    return (inputs, targets, learn_rate)
    
def get_init_cell(batch_size, rnn_size):
    def build_cell(lstm_size):
        # Use a basic LSTM cell
        lstm = tf2.compat.v1.nn.rnn_cell.BasicLSTMCell(lstm_size)
        
        # Add dropout to the cell
        drop = tf2.compat.v1.nn.rnn_cell.DropoutWrapper(lstm)#, output_keep_prob=keep_prob)
        return drop
    
    
    # Stack up multiple LSTM layers, for deep learning
    num_layers = 2
    cell = tf2.compat.v1.nn.rnn_cell.MultiRNNCell([build_cell(rnn_size) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, dtype=tf.float32)
    initial_state = tf.identity(initial_state, name="initial_state")
    return (cell, initial_state)
    
def get_embed(input_data, vocab_size, embed_dim):
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed
    
def build_rnn(cell, inputs):
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)#,initial_state=initial_state)
    final_state = tf.identity(final_state, name="final_state")
    return outputs, final_state
    
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    embed = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, embed)
    
    logits = tf2.compat.v1.layers.dense(outputs, vocab_size, activation=None)
    return logits, final_state
    
def get_batches(int_text, batch_size, seq_length):
    words_per_batch = batch_size * seq_length
    num_batch = len(int_text)//words_per_batch
    
    input_text = np.array(int_text[:(num_batch*words_per_batch)])
    target_text = np.array(int_text[1:(num_batch * words_per_batch) + 1])

    target_text[-1] = input_text[0]
    
    x_batches = np.split(input_text.reshape(batch_size, -1), num_batch, 1)
    y_batches = np.split(target_text.reshape(batch_size, -1), num_batch, 1)
    
    batches = np.array(list(zip(x_batches, y_batches)))
    return batches
    
# Number of Epochs
num_epochs = 50
# Batch Size
batch_size = 100
# RNN Size
rnn_size = 512
# Embedding Dimension Size
embed_dim = 256
# Sequence Length
seq_length = 15
# Learning Rate
learning_rate = 0.001
# Show stats for every n number of batches
show_every_n_batches = 50

save_dir = '/content/drive/MyDrive/nlp_project/models/model_1/save'

batches = get_batches(train_int_text, batch_size, seq_length)

Instructions for updating:
non-resource variables are not supported in the long term


In [8]:
!pip install tensorflow_addons
import tensorflow_addons as tfa


train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = tfa.seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)
    
#batches = get_batches(train_int_text, batch_size, seq_length)
#train_graph = tf.Graph()
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')
    
# Save parameters for checkpoint
#helper.save_params((seq_length, save_dir))

     |████████████████████████████████| 706kB 7.7MB/s 
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:702: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be "


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/legacy_tf_layers/core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Epoch   0 Batch    0/394   train_loss = 10.526
Epoch   0 Batch   50/394   train_loss = 8.302
Epoch   0 Batch  100/394   train_loss = 8.081
Epoch   0 Batch  150/394   train_loss = 8.079
Epoch   0 Batch  200/394   train_loss = 7.891
Epoch   0 Batch  250/394   train_loss = 7.989
Epoch   0 Batch  300/394   train_loss = 7.882
Epoch   0 Batch  350/394   train_loss = 7.598
Epoch   1 Batch    6/394   train_loss = 7.411
Epoch   1 Batch   56/394   train_loss = 7.441
Epoch   1 Batch  106/394   train_loss = 7.277
Epoch   1 Batch  156/394   train_loss = 7.279
Epoch   1 Batch  206/394   train_loss = 7.147
Epoch   1 Batch  256/394   train_loss = 7.167
Epoch   1 Batch  306/394   train_loss = 7.230
Epoch   1 Batch  356/394   train_loss = 6.987
Epoch   2 Batch   12/394   train_loss = 6.725
Epoch   2 Batch   62/394   train_loss = 6.826
Epoch   2 Batch  112/394   train_loss = 6.658
Epoch   2 Batch  162/394   train_loss = 6.663
Epoch   2 Batch  212/394   train_loss = 6.568
Epoch   2 Batch  262/394   train_

In [9]:
import tensorflow as tf
import numpy as np
#import helper

def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    index = np.where(probabilities == np.max(probabilities))[-1][0]

    return int_to_vocab[index]

#_,_,_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = 15, '/content/drive/MyDrive/nlp_project/models/model_1/save'
def get_tensors(loaded_graph):
    input_tensor = loaded_graph.get_tensor_by_name("input:0")
    initial_state_tensor = loaded_graph.get_tensor_by_name("initial_state:0")
    final_state_tensor = loaded_graph.get_tensor_by_name("final_state:0")
    prob_tensor = loaded_graph.get_tensor_by_name("probs:0")

    return input_tensor, initial_state_tensor, final_state_tensor, prob_tensor
    
#test_text = [int_to_vocab[word] for word in test_int_text]
#test_text = " ".join(test_text)
#test_sent_text = test_text.split("<period>")

'''
test_sentences = input("Enter sentence:")
token_dict = token_lookup()
for key, token in token_dict.items():
	test_sentences = test_sentences.replace(key, ' {} '.format(token))
test_sentences = test_sentences.lower()
test_sentences = test_sentences.split("<period>")
'''
#all_perp = []

loaded_graph = tf.Graph()
with tf.compat.v1.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.compat.v1.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)
    #print(final_state)
    # Sentences generation setup
    gen_length = 200
    prime_word = 'यही' #'moe_szyslak'
    gen_sentences = [prime_word]
            
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])
          
        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})

        #pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)

    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/nlp_project/models/model_1/save
यही एकमात्र ऐसा the only way है कि भारत का the sowing year अक्‍तूबर में a year तैयार होने के कारण 20 kilogram nitrogen fertilizer use शक्‍ति का developed use किया जाना परम आवश्यक है
इस प्रकार of कृषि in विश्व is भारत has been उत्पादन and गुण is enough after giving खेत
but this dream is known as an impromptu- to- its no way
further, the green varieties which is grown from many methods which were उपयोग by which वे could not हुआ हुए
लेवेन्डर को ही शामिल करना उचित be
भारत has reduced with small organic drugs and agriculture
apart from सरकारी समिति got प्रोत्साहन on आधार of बैंकिंग सुविधाओं s
किसानों had also ended
कृषि मंत्री ने बताया कि bonsai farmers the farmer एवं दशक में गठन a long way the size या सही जरूरत की संभावना है
हालांकि एक विशेषज्ञों ने the indian government' s माल के अनुसार the country organic agriculture the significance कम करता है
इस विधि में more output capacity प्राप्‍त होता ह